In [1]:
from make_file_for_odds_ratio import GetData
import pandas as pd
import numpy as np
from config import CONFIG

In [17]:
dfs = GetData().execute()

File already exists: reading now...
File already exists: reading now...
File already exists: reading now...


In [18]:
dfs[0].columns

Index(['SMOKE100', '_STATE', 'ASTHMA3', 'ASTHNOW', '_AGEG5YR', 'SEX',
       '_RACE_G1', 'INCOME2', '_INCOMG', 'EDUCA', '_EDUCAG', '_BMI5',
       '_BMI5CAT', 'CHILDREN', 'NUMADULT', 'HHADULT', '_CHLDCNT'],
      dtype='object')

In [19]:
mdf = pd.DataFrame()
for i in [0, 1, 2]:
    mdf = mdf.append(dfs[i])

In [20]:
def count_adults(x):
    landline = x['NUMADULT']
    cell = x['HHADULT']
    if not pd.isna(landline):
        return landline
    return cell

mdf['ADULTS'] = mdf.apply(count_adults, axis=1)

In [21]:
def add_adults_and_children(x):
    children = x['CHILDREN']
    adults = x['ADULTS']
    if not pd.isna(children) and not pd.isna(adults):
        if adults not in [77, 99] and children not in [99]:
            if children == 88:
                children = 0
            return adults + children
        elif adults not in [77, 99]:
            return adults
    elif (not pd.isna(adults)) and (adults not in [77, 99]):
        return adults
    return np.nan
    
mdf['HOUSEHOLD'] = mdf.apply(add_adults_and_children, axis=1)

In [22]:
def calculate_poverty_guideline_flag(x):
    if x['_STATE'] == 2:
        poverty_table = CONFIG.get("POVERTY").get("HOUSEHOLD_DATA").get("AK")
    elif x['_STATE'] == 15:
        poverty_table = CONFIG.get("POVERTY").get("HOUSEHOLD_DATA").get("HW")
    else:
        poverty_table = CONFIG.get("POVERTY").get("HOUSEHOLD_DATA").get("DATA")
    household = x['HOUSEHOLD']
    if not pd.isna(household):
        if household>8:
            poverty_guideline = poverty_table.get("8") + (poverty_table.get("+8") * (household-8))
        else:
            poverty_guideline = poverty_table.get(str(int(household)))
        return poverty_guideline
    return np.nan
    

mdf['POVERTY_GUIDE'] = mdf.apply(calculate_poverty_guideline_flag, axis=1)

In [23]:
mdf['POVERTY_GUIDE'].sum()

23950336070.0

In [9]:
def get_poverty_flag(x):
    income_table = CONFIG.get("POVERTY").get("INCOME_INDEX").get("DATA")
    reported_income = x['INCOME2']
    if pd.isna(reported_income) or (reported_income in [77, 99]):
        return np.nan
    income = income_table.get(str(int(reported_income)))
    if income <= x['POVERTY_GUIDE']:
        return 1
    return 0
    

mdf['POVERTY'] = mdf.apply(get_poverty_flag, axis=1)

In [10]:
mdf['POVERTY'].isna().mean()

0.16924706370495468

In [13]:
mdf[pd.isna(mdf['POVERTY'])]['INCOME2'].unique()

array([77., 99., nan])